In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import xml.etree.ElementTree as ET  
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
from statistics import mean
import math
import os
import pandas as pd
import scipy.stats as stats
import csv

In [3]:
def extract_features(path):
  features={}

  tree = ET.parse(path)
  root = tree.getroot()
  #Print Description
  #for x in root[0]:
    #print(x.text)
  #Extraction of coordinates
  a=[]
  b=[]
  all_x=[]
  all_y=[]
  x_coordinates = []
  y_coordinates = []
  for i in range(1,len(root)):
    for x in root[i]:
      if (x.text=='\n\t\t\t'):
        continue
      else:
        l = list(map(str,x.text.split(',')))
        l = [x.strip() for x in l]
        for x in l:
          xy = list(map(int,x.split(' ')))
          x_coordinates.append(xy[0])
          y_coordinates.append(xy[1])
        a.append(x_coordinates)
        b.append(y_coordinates)
        x_coordinates = []
        y_coordinates = []  
  for i in range(len(a)):
    for j in range(len(a[i])):
      all_x.append(a[i][j])
      all_y.append(b[i][j])
  
  #Time Stamp
  time_stamp = np.arange(0,len(all_x)*0.02,0.02)
  
  #Relative_x_y
  relative_x=[0]
  relative_y=[0]
  for i in range(1,len(all_x)):
    relative_x.append(all_x[i]-mean(all_x[:i]))
    relative_y.append(all_y[i]-mean(all_y[:i]))
  features['relative_x']=relative_x
  features['relative_y']=relative_y

  #Speed
  tracewise_speed=[0]
  speed=[]
  whole_speed=[]
  for i in range(len(a)):
    for j in range(1,len(a[i])):
      tracewise_speed.append(((a[i][j]-a[i][j-1])**2+(b[i][j]-b[i][j-1])**2)**(0.5)/0.02)
    speed.append(tracewise_speed)
    tracewise_speed=[0]
  for i in range(len(speed)):
    for j in range(len(speed[i])):
      whole_speed.append(speed[i][j])
  features['speed']=whole_speed

  #Speed along x and y
  speed_x=[]
  speed_y=[]
  tracewise_speed_x=[]
  tracewise_speed_y=[]
  temp_x=[0]
  temp_y=[0]
  for i in range(len(a)):
    for j in range(1,len(a[i])):
      temp_x.append((a[i][j]-a[i][j-1])/0.02)
      temp_y.append((b[i][j]-b[i][j-1])/0.02)
    tracewise_speed_x.append(temp_x)
    tracewise_speed_y.append(temp_y)
    temp_x=[0]
    temp_y=[0]
  for i in range(len(tracewise_speed_x)):
    for j in range(len(tracewise_speed_x[i])):
      speed_x.append(tracewise_speed_x[i][j])
      speed_y.append(tracewise_speed_y[i][j])
  features['speed_x'] = speed_x
  features['speed_y'] = speed_y

  #Acceleration
  acc=[]
  tracewise_acc=[0]
  whole_acc=[]
  for i in range(len(speed)):
    for j in range(1,len(speed[i])):
      tracewise_acc.append((speed[i][j]-speed[i][j-1])/0.02)
    acc.append(tracewise_acc)
    tracewise_acc=[0]
  for i in range(len(acc)):
    for j in range(len(acc[i])):
      whole_acc.append(acc[i][j])
  features['acceleration']=whole_acc

  #Acceleration along x and y
  acc_x=[0]
  acc_y=[0]
  tracewise_acceleration_x=[]
  tracewise_acceleration_y=[]
  acceleration_x=[]
  acceleration_y=[]
  for i in range(len(tracewise_speed_x)):
    for j in range(1,len(tracewise_speed_x[i])):
      acc_x.append((tracewise_speed_x[i][j]-tracewise_speed_x[i][j-1])/0.02)
      acc_y.append((tracewise_speed_y[i][j]-tracewise_speed_y[i][j-1])/0.02)
    tracewise_acceleration_x.append(acc_x)
    tracewise_acceleration_y.append(acc_y)
    acc_x=[0]
    acc_y=[0]
  for i in range(len(tracewise_acceleration_x)):
    for j in range(len(tracewise_acceleration_x[i])):
      acceleration_x.append(tracewise_acceleration_x[i][j])
      acceleration_y.append(tracewise_acceleration_y[i][j])
  features['acceleration_x']=acceleration_x
  features['acceleration_y']=acceleration_y

  #Log curvature radius
  rad=[]
  tracewise_rad=[]
  whole_rad=[]
  for i in range(len(tracewise_speed_x)):
    for j in range(len(tracewise_speed_x[i])):
      nr=(tracewise_speed_x[i][j]*tracewise_acceleration_y[i][j]-tracewise_acceleration_x[i][j]*tracewise_speed_y[i][j])
      dr=(tracewise_speed_x[i][j]**2+tracewise_speed_y[i][j]**2)**(1.5)
      if (nr==0 or dr==0):
        tracewise_rad.append(0)
      else:
        tracewise_rad.append(nr/dr)
    rad.append(tracewise_rad)
    tracewise_rad=[]
  for i in range(len(rad)):
    for j in range(len(rad[i])):
      whole_rad.append(rad[i][j])
  features['log_curvature_radius']=whole_rad

  #writing direction
  tracewise_cos=[0]
  tracewise_sin=[0]
  cos=[]
  sin=[]
  whole_cos=[]
  whole_sin=[]
  for i in range(len(a)):
    for j in range(1,len(a[i])):
      l=((a[i][j]-a[i][j-1])**2+(b[i][j]-b[i][j-1])**2)**(0.5)
      tracewise_cos.append((a[i][j]-a[i][j-1])/l)
      tracewise_sin.append((b[i][j]-b[i][j-1])/l)
    cos.append(tracewise_cos)
    sin.append(tracewise_sin)
    tracewise_cos=[0]
    tracewise_sin=[0]
  for i in range(len(cos)):
    for j in range(len(cos[i])):
      whole_cos.append(cos[i][j])
      whole_sin.append(sin[i][j])
  features['writing_direction_x']=whole_cos
  features['writing_direction_y']=whole_sin

  #Curvature
  cur_x=[]
  cur_y=[]
  tracewise_cur_x=[0]
  tracewise_cur_y=[0]
  whole_cur_x=[]
  whole_cur_y=[]
  for i in range(len(cos)):
    for j in range(1,len(cos[i])):
      tracewise_cur_x.append(np.cos(cos[i][j-1])*np.cos(cos[i][j])+np.sin(cos[i][j-1])*np.sin(cos[i][j]))
      tracewise_cur_y.append(np.cos(sin[i][j-1])*np.sin(sin[i][j])-np.sin(sin[i][j-1])*np.cos(sin[i][j]))
    cur_x.append(tracewise_cur_x)
    cur_y.append(tracewise_cur_y)
    tracewise_cur_x=[0]
    tracewise_cur_y=[0]
  for i in range(len(cur_x)):
    for j in range(len(cur_x[i])):
      whole_cur_x.append(cur_x[i][j])
      whole_cur_y.append(cur_y[i][j])
  features['curvature_x']=whole_cur_x
  features['curvature_y']=whole_cur_y

  #Vicinity Aspect
  temp_x=[]
  temp_y=[]
  left_x=[]
  right_x=[]
  left_y=[]
  right_y=[]
  var=[]
  vc=[]
  vl=[]
  vs_cos=[]
  vs_sin=[]
  tracewise_var=[]
  for i in range(len(a)):
    for j in range(len(a[i])):
      if len(a[i])>=11:
        if len(a[i][:j])>5:
          left_x=a[i][j-5:j]
        elif len(a[i][:j])<=5:
          left_x=a[i][:j]
        if len(a[i][j+1:])>5:
          right_x=a[i][j+1:j+6]
        elif len(a[i][j+1:])<=5:
          right_x=a[i][j+1:]
      elif len(a[i])<11:
        if len(a[i][:j])>5:
          left_x=a[i][j-5:j]
        elif len(a[i][:j])<=5:
          left_x=a[i][:j]
        if len(a[i][j+1:])>5:
          right_x=a[i][j+1:j+6]
        elif len(a[i][j+1:])<=5:
          right_x=a[i][j+1:]

      if len(b[i])>=11:
        if len(b[i][:j])>5:
          left_y=b[i][j-5:j]
        elif len(b[i][:j])<=5:
          left_y=b[i][:j]
        if len(b[i][j+1:])>5:
          right_y=b[i][j+1:j+6]
        elif len(b[i][j+1:])<=5:
          right_y=b[i][j+1:]
      elif len(b[i])<11:
        if len(b[i][:j])>5:
          left_y=b[i][j-5:j]
        elif len(b[i][:j])<=5:
          left_y=b[i][:j]
        if len(b[i][j+1:])>5:
          right_y=b[i][j+1:j+6]
        elif len(b[i][j+1:])<=5:
          right_y=b[i][j+1:]

      left_x.append(a[i][j])
      left_y.append(b[i][j])
      temp_x=left_x+right_x
      temp_y=left_y+right_y
      dx=max(temp_x)-min(temp_x)
      dy=max(temp_y)-min(temp_y)
      if dx+dy==0:
        var.append(0)
      else:
        var.append((dy-dx)/(dx+dy))
      
      #vicinity_curliness
      dist=[]
      for k in range(1,len(temp_x)):
        dist.append(((temp_x[k]-temp_x[k-1])**2+(temp_y[k]-temp_y[k-1])**2)**(0.5))
      if dx+dy==0:
        vc.append(0)
      else:
        vc.append(sum(dist)/max(dx,dy))
      dist=[]

      #vicinity linearity
      if temp_x[-1]==temp_x[0]:
        slope = 'vertical_line'
      else:
        slope = (temp_y[-1]-temp_y[0])/(temp_x[-1]-temp_x[0])
      dist=[]
      for k in range(len(temp_x)):
        if slope=='vertical_line':
          dist.append(0)
        else:
          c=temp_y[0]-slope*temp_x[0]
          dist.append(((abs(slope*temp_x[k]-1*temp_y[k]+c))/(slope**2+1)**(0.5))**2)
      
      vl.append(mean(dist))
      dist=[]

      #vicinity slope
      if (temp_x[-1]-temp_x[0])==0:
        vs_cos.append(1)
        vs_sin.append(0)
      else:
        slope = (temp_y[-1]-temp_y[0])/(temp_x[-1]-temp_x[0])
        theta = math.atan(slope)
        vs_cos.append(math.sin(theta))
        vs_sin.append(math.cos(theta))

  features['vicinity_aspect']=var
  features['vicinity_curliness']=vc
  features['vicinity_linearity']=vl
  features['vicinity_slope_x']=vs_cos
  features['vicinity_slope_y']=vs_sin

  return(features)

In [4]:
def normalization(fea):
  fea_norm={}
  rex = fea['relative_x'];
  fea_norm["relative_x"] = stats.zscore(rex)
  rey = fea['relative_y'];
  fea_norm["relative_y"] = stats.zscore(rey)
  sp=fea['speed'];
  fea_norm["speed"] = stats.zscore(sp)
  sp_x = fea['speed_x'];
  fea_norm["speed_x"] = stats.zscore(sp_x)
  sp_y = fea['speed_y'];
  fea_norm["speed_y"] = stats.zscore(sp_y)
  acc = fea['acceleration'];
  fea_norm["acceleration"] = stats.zscore(acc)
  acc_x = fea['acceleration_x']
  fea_norm['acceleration_x'] = stats.zscore(acc_x)
  acc_y = fea['acceleration_y']
  fea_norm['acceleration_y'] = stats.zscore(acc_y)
  lcr = fea['log_curvature_radius'];
  fea_norm["log_curvature_radus"] = stats.zscore(lcr)
  wdx = fea['writing_direction_x'];
  fea_norm["writing_direction_x"] = stats.zscore(wdx)
  wdy = fea['writing_direction_y'];
  fea_norm["writing_direction_y"] = stats.zscore(wdy)
  cx = fea['curvature_x'];
  fea_norm["curvature_x"] = stats.zscore(cx)
  cy = fea['curvature_y'];
  fea_norm["curvature_y"] = stats.zscore(cy)
  va = fea['vicinity_aspect'];
  fea_norm["viscinity_aspect"] = stats.zscore(va)
  vc = fea['vicinity_curliness'];
  fea_norm["viscinity_curliness"] = stats.zscore(vc)
  vl = fea['vicinity_linearity'];
  fea_norm["viscinity_linearty"] = stats.zscore(vl)
  vsx = fea['vicinity_slope_x'];
  fea_norm["viscinity_slope_x"] = stats.zscore(vsx)
  vsy = fea['vicinity_slope_y'];
  fea_norm["viscinity_slope_y"] = stats.zscore(vsy)
  
  return fea_norm


In [5]:
writer_path='/content/drive/MyDrive/Datasets/Split_6_4/Train/1'

In [6]:
with open('/content/drive/MyDrive/Datasets/Split_6_4/Features_Train/1.csv', 'w') as f:
  csvwriter = csv.writer(f)

  a=[[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[]]
  for sample in os.listdir(writer_path):
    sample_path = os.path.join(writer_path,sample)
    print(sample_path)
    features = extract_features(sample_path)
    normalized = normalization(features)
    c=0
    for key, values in normalized.items():
      a[c].extend(normalized[key])
      c+=1
  for x in a:
    csvwriter.writerow(x)

/content/drive/MyDrive/Datasets/Split_6_4/Train/1/62.inkml
/content/drive/MyDrive/Datasets/Split_6_4/Train/1/6.inkml
/content/drive/MyDrive/Datasets/Split_6_4/Train/1/46.inkml
/content/drive/MyDrive/Datasets/Split_6_4/Train/1/48.inkml
/content/drive/MyDrive/Datasets/Split_6_4/Train/1/65.inkml
/content/drive/MyDrive/Datasets/Split_6_4/Train/1/56.inkml
/content/drive/MyDrive/Datasets/Split_6_4/Train/1/68.inkml
/content/drive/MyDrive/Datasets/Split_6_4/Train/1/9.inkml
/content/drive/MyDrive/Datasets/Split_6_4/Train/1/8.inkml
/content/drive/MyDrive/Datasets/Split_6_4/Train/1/28.inkml
/content/drive/MyDrive/Datasets/Split_6_4/Train/1/50.inkml
/content/drive/MyDrive/Datasets/Split_6_4/Train/1/18.inkml
/content/drive/MyDrive/Datasets/Split_6_4/Train/1/30.inkml
/content/drive/MyDrive/Datasets/Split_6_4/Train/1/22.inkml
/content/drive/MyDrive/Datasets/Split_6_4/Train/1/54.inkml
/content/drive/MyDrive/Datasets/Split_6_4/Train/1/25.inkml
/content/drive/MyDrive/Datasets/Split_6_4/Train/1/52.inkml


In [ ]:
writer_path='/content/drive/MyDrive/Datasets/Split_6_4/Train/3'

In [ ]:
with open('/content/drive/MyDrive/Datasets/Split_6_4/Features_Train/3.csv', 'w') as f:
  csvwriter = csv.writer(f)

  a=[[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[]]
  for sample in os.listdir(writer_path):
    sample_path = os.path.join(writer_path,sample)
    print(sample_path)
    features = extract_features(sample_path)
    normalized = normalization(features)
    c=0
    for key, values in normalized.items():
      a[c].extend(normalized[key])
      c+=1
  for x in a:
    csvwriter.writerow(x)

/content/drive/MyDrive/Datasets/Split_6_4/Train/3/46.inkml
/content/drive/MyDrive/Datasets/Split_6_4/Train/3/3.inkml
/content/drive/MyDrive/Datasets/Split_6_4/Train/3/6.inkml
/content/drive/MyDrive/Datasets/Split_6_4/Train/3/72.inkml
/content/drive/MyDrive/Datasets/Split_6_4/Train/3/60.inkml
/content/drive/MyDrive/Datasets/Split_6_4/Train/3/56.inkml
/content/drive/MyDrive/Datasets/Split_6_4/Train/3/41.inkml
/content/drive/MyDrive/Datasets/Split_6_4/Train/3/45.inkml
/content/drive/MyDrive/Datasets/Split_6_4/Train/3/21.inkml
/content/drive/MyDrive/Datasets/Split_6_4/Train/3/1.inkml
/content/drive/MyDrive/Datasets/Split_6_4/Train/3/65.inkml
/content/drive/MyDrive/Datasets/Split_6_4/Train/3/75.inkml
/content/drive/MyDrive/Datasets/Split_6_4/Train/3/38.inkml
/content/drive/MyDrive/Datasets/Split_6_4/Train/3/54.inkml
/content/drive/MyDrive/Datasets/Split_6_4/Train/3/50.inkml
/content/drive/MyDrive/Datasets/Split_6_4/Train/3/66.inkml
/content/drive/MyDrive/Datasets/Split_6_4/Train/3/13.inkml


In [ ]:
writer_path='/content/drive/MyDrive/Datasets/Split_6_4/Train/4'

In [ ]:
with open('/content/drive/MyDrive/Datasets/Split_6_4/Features_Train/4.csv', 'w') as f:
  csvwriter = csv.writer(f)

  a=[[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[]]
  for sample in os.listdir(writer_path):
    sample_path = os.path.join(writer_path,sample)
    print(sample_path)
    features = extract_features(sample_path)
    normalized = normalization(features)
    c=0
    for key, values in normalized.items():
      a[c].extend(normalized[key])
      c+=1
  for x in a:
    csvwriter.writerow(x)

/content/drive/MyDrive/Datasets/Split_6_4/Train/4/64.inkml
/content/drive/MyDrive/Datasets/Split_6_4/Train/4/17.inkml
/content/drive/MyDrive/Datasets/Split_6_4/Train/4/37.inkml
/content/drive/MyDrive/Datasets/Split_6_4/Train/4/26.inkml
/content/drive/MyDrive/Datasets/Split_6_4/Train/4/40.inkml
/content/drive/MyDrive/Datasets/Split_6_4/Train/4/19.inkml
/content/drive/MyDrive/Datasets/Split_6_4/Train/4/63.inkml
/content/drive/MyDrive/Datasets/Split_6_4/Train/4/31.inkml
/content/drive/MyDrive/Datasets/Split_6_4/Train/4/34.inkml
/content/drive/MyDrive/Datasets/Split_6_4/Train/4/21.inkml
/content/drive/MyDrive/Datasets/Split_6_4/Train/4/24.inkml
/content/drive/MyDrive/Datasets/Split_6_4/Train/4/32.inkml
/content/drive/MyDrive/Datasets/Split_6_4/Train/4/1.inkml
/content/drive/MyDrive/Datasets/Split_6_4/Train/4/16.inkml
/content/drive/MyDrive/Datasets/Split_6_4/Train/4/39.inkml
/content/drive/MyDrive/Datasets/Split_6_4/Train/4/30.inkml
/content/drive/MyDrive/Datasets/Split_6_4/Train/4/25.inkm

In [ ]:
writer_path='/content/drive/MyDrive/Datasets/Split_6_4/Train/5'

In [ ]:
with open('/content/drive/MyDrive/Datasets/Split_6_4/Features_Train/5.csv', 'w') as f:
  csvwriter = csv.writer(f)

  a=[[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[]]
  for sample in os.listdir(writer_path):
    sample_path = os.path.join(writer_path,sample)
    print(sample_path)
    features = extract_features(sample_path)
    normalized = normalization(features)
    c=0
    for key, values in normalized.items():
      a[c].extend(normalized[key])
      c+=1
  for x in a:
    csvwriter.writerow(x)

/content/drive/MyDrive/Datasets/Split_6_4/Train/5/50.inkml
/content/drive/MyDrive/Datasets/Split_6_4/Train/5/12.inkml
/content/drive/MyDrive/Datasets/Split_6_4/Train/5/67.inkml
/content/drive/MyDrive/Datasets/Split_6_4/Train/5/58.inkml
/content/drive/MyDrive/Datasets/Split_6_4/Train/5/3.inkml
/content/drive/MyDrive/Datasets/Split_6_4/Train/5/40.inkml
/content/drive/MyDrive/Datasets/Split_6_4/Train/5/30.inkml
/content/drive/MyDrive/Datasets/Split_6_4/Train/5/26.inkml
/content/drive/MyDrive/Datasets/Split_6_4/Train/5/10.inkml
/content/drive/MyDrive/Datasets/Split_6_4/Train/5/38.inkml
/content/drive/MyDrive/Datasets/Split_6_4/Train/5/29.inkml
/content/drive/MyDrive/Datasets/Split_6_4/Train/5/5.inkml
/content/drive/MyDrive/Datasets/Split_6_4/Train/5/27.inkml
/content/drive/MyDrive/Datasets/Split_6_4/Train/5/37.inkml
/content/drive/MyDrive/Datasets/Split_6_4/Train/5/43.inkml
/content/drive/MyDrive/Datasets/Split_6_4/Train/5/52.inkml
/content/drive/MyDrive/Datasets/Split_6_4/Train/5/14.inkml

In [ ]:
writer_path='/content/drive/MyDrive/Datasets/Split_6_4/Train/6'

In [ ]:
with open('/content/drive/MyDrive/Datasets/Split_6_4/Features_Train/6.csv', 'w') as f:
  csvwriter = csv.writer(f)

  a=[[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[]]
  for sample in os.listdir(writer_path):
    sample_path = os.path.join(writer_path,sample)
    print(sample_path)
    features = extract_features(sample_path)
    normalized = normalization(features)
    c=0
    for key, values in normalized.items():
      a[c].extend(normalized[key])
      c+=1
  for x in a:
    csvwriter.writerow(x)

/content/drive/MyDrive/Datasets/Split_6_4/Train/6/33.inkml
/content/drive/MyDrive/Datasets/Split_6_4/Train/6/32.inkml
/content/drive/MyDrive/Datasets/Split_6_4/Train/6/37.inkml
/content/drive/MyDrive/Datasets/Split_6_4/Train/6/43.inkml
/content/drive/MyDrive/Datasets/Split_6_4/Train/6/52.inkml
/content/drive/MyDrive/Datasets/Split_6_4/Train/6/2.inkml
/content/drive/MyDrive/Datasets/Split_6_4/Train/6/45.inkml
/content/drive/MyDrive/Datasets/Split_6_4/Train/6/48.inkml
/content/drive/MyDrive/Datasets/Split_6_4/Train/6/6.inkml
/content/drive/MyDrive/Datasets/Split_6_4/Train/6/16.inkml
/content/drive/MyDrive/Datasets/Split_6_4/Train/6/30.inkml
/content/drive/MyDrive/Datasets/Split_6_4/Train/6/23.inkml
/content/drive/MyDrive/Datasets/Split_6_4/Train/6/36.inkml
/content/drive/MyDrive/Datasets/Split_6_4/Train/6/38.inkml
/content/drive/MyDrive/Datasets/Split_6_4/Train/6/19.inkml
/content/drive/MyDrive/Datasets/Split_6_4/Train/6/31.inkml
/content/drive/MyDrive/Datasets/Split_6_4/Train/6/40.inkml

In [ ]:
writer_path='/content/drive/MyDrive/Datasets/Split_6_4/Train/7'

In [ ]:
with open('/content/drive/MyDrive/Datasets/Split_6_4/Features_Train/7.csv', 'w') as f:
  csvwriter = csv.writer(f)

  a=[[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[]]
  for sample in os.listdir(writer_path):
    sample_path = os.path.join(writer_path,sample)
    print(sample_path)
    features = extract_features(sample_path)
    normalized = normalization(features)
    c=0
    for key, values in normalized.items():
      a[c].extend(normalized[key])
      c+=1
  for x in a:
    csvwriter.writerow(x)

/content/drive/MyDrive/Datasets/Split_6_4/Train/7/9.inkml
/content/drive/MyDrive/Datasets/Split_6_4/Train/7/29.inkml
/content/drive/MyDrive/Datasets/Split_6_4/Train/7/3.inkml
/content/drive/MyDrive/Datasets/Split_6_4/Train/7/22.inkml
/content/drive/MyDrive/Datasets/Split_6_4/Train/7/15.inkml
/content/drive/MyDrive/Datasets/Split_6_4/Train/7/24.inkml
/content/drive/MyDrive/Datasets/Split_6_4/Train/7/41.inkml
/content/drive/MyDrive/Datasets/Split_6_4/Train/7/4.inkml
/content/drive/MyDrive/Datasets/Split_6_4/Train/7/48.inkml
/content/drive/MyDrive/Datasets/Split_6_4/Train/7/33.inkml
/content/drive/MyDrive/Datasets/Split_6_4/Train/7/13.inkml
/content/drive/MyDrive/Datasets/Split_6_4/Train/7/12.inkml
/content/drive/MyDrive/Datasets/Split_6_4/Train/7/45.inkml
/content/drive/MyDrive/Datasets/Split_6_4/Train/7/47.inkml
/content/drive/MyDrive/Datasets/Split_6_4/Train/7/6.inkml
/content/drive/MyDrive/Datasets/Split_6_4/Train/7/34.inkml
/content/drive/MyDrive/Datasets/Split_6_4/Train/7/37.inkml
/